In [12]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import BertModel
from tokenizers import BertWordPieceTokenizer

import torch
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader, Dataset

import pandas as pd
from tqdm import tqdm

from sklearn.metrics import classification_report, confusion_matrix

In [2]:
device = None

import torch
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print (device)
else:
    print ("MPS device not found.")

mps


In [3]:
# 모델 및 토크나이저 로드
model = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [4]:
# test 데이터 가져오기
test_data = "./yahoo_answers_csv/test.csv"
test_df = pd.read_csv(test_data)

sentence = test_df.iloc[0, 1] + " " + test_df.iloc[0, 2] + " " + test_df.iloc[0, 3]
true_label = test_df.iloc[0, 0]
print(f"sentence: {sentence}")

inputs = tokenizer(sentence, return_tensors="pt").to(device)

token_length = inputs.input_ids.shape[1]
print(f"token length: {token_length}\n")



# 모델에 입력값 넣기
outputs = model(**inputs)

predictions = outputs.logits.argmax(dim=-1)
print(f"pred output: {predictions.item() + 1}")
print(f"true label: {true_label}")
print(outputs.logits)



"""
class 정보

1: Society & Culture
2: Science & Mathematics
3: Health
4: Education & Reference
5: Computers & Internet
6: Sports
7: Business & Finance
8: Entertainment & Music
9: Family & Relationships
10: Politics & Government

"""


sentence: Why does Zebras have stripes? What is the purpose or those stripes? Who do they serve the Zebras in the wild life? this provides camouflage - predator vision is such that it is usually difficult for them to see complex patterns
token length: 48

pred output: 2
true label: 2
tensor([[ 0.7546,  3.1281, -0.6842,  2.3181, -1.2996, -0.6507,  1.6057, -1.0207,
         -1.7158, -0.7543]], device='mps:0', grad_fn=<LinearBackward0>)


'\nclass 정보\n\n1: Society & Culture\n2: Science & Mathematics\n3: Health\n4: Education & Reference\n5: Computers & Internet\n6: Sports\n7: Business & Finance\n8: Entertainment & Music\n9: Family & Relationships\n10: Politics & Government\n\n'

In [5]:
class TestDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.sentences = df.iloc[:, 1:].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).values
        self.labels = df.iloc[:, 0].values
        self.tokenizer = tokenizer
  
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        inputs = self.tokenizer(sentence, truncation=True, max_length=512, padding='max_length', return_tensors="pt")
        label = torch.tensor(self.labels[idx]) # label은 1부터 시작하기 때문에, 나중에 inference할 때에 예측값에 1을 더해줘야 합니다.
        
        return inputs, label

In [6]:
test_data = "./yahoo_answers_csv/test.csv"
test_df = pd.read_csv(test_data, header=None)

test_dataset = TestDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [14]:
# 1번째 레이어 1번 헤드 끄기

# print(model.bert.encoder.layer[0].attention.self.value.weight.data)
# model.bert.encoder.layer[0].attention.self.value.weight.data[:, :64] = 0
# print(model.bert.encoder.layer[0].attention.self.value.weight.data)

In [19]:
for layer_index in range(12):
    for head_index in range(12):

        model = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
        model = model.to(device)

        # print(model.bert.encoder.layer[layer_index].attention.self.value.weight.data)
        model.bert.encoder.layer[layer_index].attention.self.value.weight.data[head_index*64:(head_index+1)*64, :] = 0
        # print(model.bert.encoder.layer[layer_index].attention.self.value.weight.data)

        preds = []
        true_labels = []

        for batch in tqdm(test_loader, desc="Evaluating"):
            inputs, labels = batch
            inputs = {k: v.squeeze(1).to(device) for k, v in inputs.items()} 
            labels = labels.to(device)

            with torch.no_grad():
                outputs = model(**inputs)
            prediction = outputs.logits.argmax(dim=-1) + 1
            
            preds.extend(prediction.tolist())
            true_labels.extend(labels.tolist())

        print(f"off attention head index // layer: {layer_index+1}, head: {head_index+1}")
            
        print('classification report')
        print(classification_report(true_labels, preds, digits=5))

        print('confusion matrix')
        print(confusion_matrix(true_labels, preds))

Evaluating: 100%|██████████| 1875/1875 [22:57<00:00,  1.36it/s]


off attention head index // layer: 1, head: 1
classification report
              precision    recall  f1-score   support

           1    0.62009   0.61833   0.61921      6000
           2    0.73934   0.80317   0.76993      6000
           3    0.74787   0.86517   0.80226      6000
           4    0.65157   0.54200   0.59176      6000
           5    0.85083   0.88317   0.86670      6000
           6    0.91874   0.91200   0.91536      6000
           7    0.76982   0.41750   0.54139      6000
           8    0.71085   0.79367   0.74998      6000
           9    0.72584   0.82733   0.77327      6000
          10    0.76195   0.83700   0.79771      6000

    accuracy                        0.74993     60000
   macro avg    0.74969   0.74993   0.74276     60000
weighted avg    0.74969   0.74993   0.74276     60000

confusion matrix
[[3710  127  255  405   62   42   90  380  636  293]
 [ 162 4819  410  296   60   43   64   69   17   60]
 [ 157  103 5191   71    9   35   24   81  279   5

Evaluating: 100%|██████████| 1875/1875 [22:50<00:00,  1.37it/s]


off attention head index // layer: 1, head: 2
classification report
              precision    recall  f1-score   support

           1    0.61858   0.62383   0.62119      6000
           2    0.73720   0.80650   0.77030      6000
           3    0.74870   0.86500   0.80266      6000
           4    0.65031   0.54117   0.59074      6000
           5    0.85231   0.88200   0.86690      6000
           6    0.92168   0.90817   0.91488      6000
           7    0.77317   0.41300   0.53840      6000
           8    0.70837   0.79267   0.74815      6000
           9    0.72568   0.82667   0.77289      6000
          10    0.76219   0.83650   0.79762      6000

    accuracy                        0.74955     60000
   macro avg    0.74982   0.74955   0.74237     60000
weighted avg    0.74982   0.74955   0.74237     60000

confusion matrix
[[3743  123  253  404   59   36   82  381  635  284]
 [ 162 4839  403  292   55   40   59   71   19   60]
 [ 159  109 5190   69    9   35   23   80  276   5

Evaluating: 100%|██████████| 1875/1875 [22:49<00:00,  1.37it/s]


...

Evaluating: 100%|██████████| 1875/1875 [22:58<00:00,  1.36it/s]


off attention head index // layer: 5, head: 6
classification report
              precision    recall  f1-score   support

           1    0.62032   0.61867   0.61949      6000
           2    0.73899   0.80550   0.77081      6000
           3    0.75138   0.86233   0.80304      6000
           4    0.65144   0.54200   0.59170      6000
           5    0.85041   0.88400   0.86688      6000
           6    0.91987   0.91067   0.91524      6000
           7    0.77194   0.41633   0.54093      6000
           8    0.70921   0.79467   0.74951      6000
           9    0.72212   0.82767   0.77130      6000
          10    0.76173   0.83600   0.79714      6000

    accuracy                        0.74978     60000
   macro avg    0.74974   0.74978   0.74260     60000
weighted avg    0.74974   0.74978   0.74260     60000

confusion matrix
[[3712  126  250  408   60   39   85  385  647  288]
 [ 166 4833  397  292   58   40   59   73   19   63]
 [ 154  112 5174   74    9   38   23   81  284   5

Evaluating: 100%|██████████| 1875/1875 [22:54<00:00,  1.36it/s]


off attention head index // layer: 5, head: 7
classification report
              precision    recall  f1-score   support

           1    0.61849   0.62117   0.61982      6000
           2    0.73964   0.80633   0.77155      6000
           3    0.75211   0.86217   0.80339      6000
           4    0.65391   0.53817   0.59042      6000
           5    0.84827   0.88517   0.86632      6000
           6    0.91937   0.91033   0.91483      6000
           7    0.77222   0.41417   0.53916      6000
           8    0.71127   0.79283   0.74984      6000
           9    0.72176   0.82967   0.77196      6000
          10    0.75968   0.83717   0.79654      6000

    accuracy                        0.74972     60000
   macro avg    0.74967   0.74972   0.74238     60000
weighted avg    0.74967   0.74972   0.74238     60000

confusion matrix
[[3727  122  247  397   59   41   87  380  649  291]
 [ 168 4838  392  289   59   41   56   73   19   65]
 [ 158  109 5173   70    9   39   24   80  285   5

Evaluating:  43%|████▎     | 808/1875 [09:53<12:42,  1.40it/s]

In [ ]:
preds = []
true_labels = []

for i in range(len(test_df)):
    # 문장과 정답 레이블 가져오기
    sentence = str(test_df.iloc[i, 1]) + " " + str(test_df.iloc[i, 2]) + " " + str(test_df.iloc[i, 3])
    true_label = test_df.iloc[i, 0]
    
    # 문장을 모델의 입력 형식으로 변환
    inputs = tokenizer(sentence, truncation=True, max_length=512, padding='max_length', return_tensors="pt").to(device)

    # 모델에 입력값 넣기
    outputs = model(**inputs)

    # 모델의 예측 결과를 가져오기
    prediction = outputs.logits.argmax(dim=-1).item() + 1 # prediction은 0부터 시작하기 때문에 1을 더해줍니다.
    
    # 예측 결과와 정답 레이블을 리스트에 추가
    preds.append(prediction)
    true_labels.append(true_label)

    if (i+1) % 100 == 0:
        print(f"{i+1}th done.")

# 각 클래스별 정확도 계산 및 출력
print(classification_report(true_labels, preds))